# Pride Results Analysis
This notebook is used to analyze the results file from PRIDE-PPPAR


It assumes you have generated daily rinex and already run PRIDE on those files


In [1]:
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [2]:
#Select a site and survey

# network='aleutian'
# site='SEM1'
# survey='2022_A_1049'

network='cascadia-gorda'
site='NCC1'
survey='2022_A_1065'
#survey='2023_A_1063'


#point this at where you have the pride results
pride_dir = f"/Users/gottlieb/working/GIT/seafloor_geodesy_notebooks/notebooks/data/Pride/"


#rinex_dir = f"/Users/gottlieb/working/GIT/seafloor_geodesy_notebooks/notebooks/data/{network}/{site}/{survey}/raw/"
#rinex_filename = f"NCC1{dayOfYear}0.23o"


# dayOfYear = 129
# year = survey.split('_')[0]
# log_path = os.path.join(pride_dir, f"log_{year}{dayOfYear}_{site}")
# kin_path = os.path.join(pride_dir, f"kin_{year}{dayOfYear}_{site}")
# res_path = os.path.join(pride_dir, f"res_{year}{dayOfYear}_{site}")



# check LOG file for epochs used vs removed

In [3]:

def read_log_header(log_path):
    log_header = {}
    with open(log_path, "r") as file:
        for i, line in enumerate(file):
            if "END OF HEADER" in line:
                end_of_header = i
                break
            log_header[line.rstrip()[60:]] = line.rstrip()[:60].strip()
    return log_header

def get_epochs_from_log(log_path):
    log_header = read_log_header(log_path)
    #print(log_header)
    epo_ava = log_header['EPO AVA/REM/NEW'].split()[0]
    epo_rem = log_header['EPO AVA/REM/NEW'].split()[1]
    print(f"Epochs used: {epo_ava}")
    print(f"Epochs removed: {epo_rem}")
    print(f"Percent used: {round(int(epo_ava)/(int(epo_ava)+int(epo_rem))*100,2)}")

#get_epochs_from_log(log_path)

# Parse and plot the KIN file (adding wrms from RES file)

In [4]:
def read_kin_header(kin_path):
    kin_header = {}
    with open(kin_path, "r") as kin_file:
        for i, line in enumerate(kin_file):
            if "END OF HEADER" in line:
                end_of_header = i
                break
            kin_header[line.rstrip()[60:]] = line.rstrip()[:60].strip()
    return kin_header

def parse_kin_header(kin_path):
    kin_header = read_kin_header(kin_path)
    keys = ['OBS STRICT EDITING','OBS FIRST EPOCH','OBS LAST EPOCH','OBS INTERVAL (sec)','OBS MASK ANGLE (deg)',
            'MEASUREMENT NOISE PSEUDORANGE (1-SIGMA, meter)','MEASUREMENT NOISE CARRIER PHASE (1-SIGMA, cycle)',
            'SITE ANTENNA TYPE','SITE ANTENNA E/N/H (meter)','AMB FIXING']
    for key in kin_header:
        if key in keys:
            print(key, kin_header[key])

def read_kin_data(kin_path):
    with open(kin_path, "r") as kin_file:
            for i, line in enumerate(kin_file):
                if "END OF HEADER" in line:
                    end_of_header = i + 1
                    break
    cols = ["Mjd","Sod","*","X","Y","Z","Latitude","Longitude","Height","Nsat","G","R","E","C2","C3","J","PDOP"]
    colspecs = [(0,6),(6,16),(16,18),(18,32),(32,46),(46,60),(60,77),(77,94),(94,108),
                (108,114),(114,117),(117,120),(120,123),(123,126),(126,129),(129,132),(132,140)]
    kin_df = pd.read_fwf(kin_path, header=end_of_header, colspecs=colspecs, names=cols, on_bad_lines='skip')
    #kin_df = pd.read_csv(kin_path, sep="\s+", names=cols, header=end_of_header, on_bad_lines='skip')
    kin_df.set_index(pd.to_datetime(kin_df['Mjd']+2400000.5, unit='D',origin="julian")+pd.to_timedelta(kin_df['Sod'], unit='sec'), inplace=True)
    return kin_df

#read res and caculate wrms
def get_wrms_from_res(res_path):
    with open(res_path, "r") as res_file:
        timestamps = []
        data = []
        wrms = 0
        sumOfSquares = 0
        sumOfWeights = 0
        line = res_file.readline() #first line is header and we can throw away
        while True:
            if line == "": #break at EOF
                break
            line_data = line.split()
            if line_data[0] == 'TIM': #for a given epoch
                sumOfSquares = 0
                sumOfWeights = 0
                #parse date fields and make a timestamp
                seconds = float(line_data[6])
                SS = int(seconds)
                f = str(seconds - SS).split('.')[-1]
                isodate = f"{line_data[1]}-{line_data[2].zfill(2)}-{line_data[3].zfill(2)}T{line_data[4].zfill(2)}:{line_data[5].zfill(2)}:{str(SS).zfill(2)}.{f}"
                timestamp = datetime.fromisoformat(isodate)
                timestamps.append(timestamp)
                #loop through SV data for that epoch, stop at next timestamp
                line = res_file.readline()
                line_data = line.split()
                while not line.startswith('TIM'):
                    phase_residual = float(line_data[1])
                    phase_weight = float(line_data[3].replace('D','E'))
                    sumOfSquares += phase_residual**2 * phase_weight
                    sumOfWeights += phase_weight
                    line = res_file.readline()
                    if line == "":
                        break
                    line_data = line.split()
                wrms = (sumOfSquares/sumOfWeights)**0.5 * 1000 #in mm
                data.append(wrms)
            else:
                line = res_file.readline()
    res_df = pd.DataFrame({'date':timestamps,'wrms':data}).set_index('date')
    return res_df

#parse_kin_header(kin_path)
#kin_df = read_kin_data(kin_path)
#wrms_df = get_wrms_from_res(res_path)
#kin_df['wrms']=wrms_df['wrms']

In [5]:
def plot_kin_results(kin_df, title=None, save_as=None):
    #use this one if you dont have wrms data
    size=3
    bad_nsat = kin_df[kin_df['Nsat']<=4]
    bad_pdop = kin_df[kin_df['PDOP']>=5]
    #bad_wrms = kin_df[kin_df['wrms']>=20]
    fig, axs = plt.subplots(5,1, figsize=(10,10), sharex=True, )
    axs[0].scatter(kin_df.index,kin_df['Latitude'],s=size)
    axs[0].set_ylabel('Latitude')
    axs[1].scatter(kin_df.index,kin_df['Longitude'],s=size)
    axs[1].set_ylabel('Longitude')
    axs[2].scatter(kin_df.index,kin_df['Height'],s=size)
    axs[2].set_ylabel('Height')
    axs[3].scatter(kin_df.index,kin_df['Nsat'],s=size)
    axs[3].scatter(bad_nsat.index,bad_nsat['Nsat'],s=size*2, color='red')
    axs[3].set_ylabel('Nsat')
    axs[4].scatter(kin_df.index,kin_df['PDOP'],s=size)
    axs[4].scatter(bad_pdop.index,bad_pdop['PDOP'],s=size*2, color='red')
    axs[4].set_ylabel('log PDOP')
    axs[4].set_yscale('log')
    axs[4].set_ylim(1,100)
    
    axs[0].ticklabel_format(axis='y',useOffset=False, style='plain')
    axs[1].ticklabel_format(axis='y',useOffset=False, style='plain')
    for ax in axs:
        ax.grid(True,c='lightgrey',zorder=0,lw=1,ls=':')
    plt.xticks(rotation=70)
    fig.suptitle(f"PRIDE-PPPAR results for {os.path.basename(title)}")
    fig.tight_layout()
    if save_as is not None:
        plt.savefig(save_as)
    plt.close()
    print(f"epochs with nsat <= 4: {round(len(bad_nsat)/len(kin_df)*100,2)} %")
    print(f"epochs with PDOP >= 5: {round(len(bad_pdop)/len(kin_df)*100,2)} %")
    #print(f"epochs with wrms >= 20: {round(len(bad_wrms)/len(kin_df)*100,2)} %")

def plot_kin_results_wrms(kin_df, title=None, save_as=None):
    #use this one if you have wrms data
    size=3
    bad_nsat = kin_df[kin_df['Nsat']<=4]
    bad_pdop = kin_df[kin_df['PDOP']>=5]
    bad_wrms = kin_df[kin_df['wrms']>=20]
    fig, axs = plt.subplots(6,1, figsize=(10,10), sharex=True, )
    axs[0].scatter(kin_df.index,kin_df['Latitude'],s=size)
    axs[0].set_ylabel('Latitude')
    axs[1].scatter(kin_df.index,kin_df['Longitude'],s=size)
    axs[1].set_ylabel('Longitude')
    axs[2].scatter(kin_df.index,kin_df['Height'],s=size)
    axs[2].set_ylabel('Height')
    axs[3].scatter(kin_df.index,kin_df['Nsat'],s=size)
    axs[3].scatter(bad_nsat.index,bad_nsat['Nsat'],s=size*2, color='red')
    axs[3].set_ylabel('Nsat')
    axs[4].scatter(kin_df.index,kin_df['PDOP'],s=size)
    axs[4].scatter(bad_pdop.index,bad_pdop['PDOP'],s=size*2, color='red')
    axs[4].set_ylabel('log PDOP')
    axs[4].set_yscale('log')
    axs[4].set_ylim(1,100)
    axs[5].scatter(kin_df.index,kin_df['wrms'],s=size)
    #axs[5].plot(kin_df['wrms'])
    #axs[5].scatter(bad_wrms.index,bad_wrms['wrms'],s=size*2, color='red')
    axs[5].set_ylabel('wrms mm')
    #axs[5].set_yscale('log')
    #axs[5].set_ylim(0,100)

    axs[0].ticklabel_format(axis='y',useOffset=False, style='plain')
    axs[1].ticklabel_format(axis='y',useOffset=False, style='plain')
    for ax in axs:
        ax.grid(True,c='lightgrey',zorder=0,lw=1,ls=':')
    plt.xticks(rotation=70)
    fig.suptitle(f"PRIDE-PPPAR results for {os.path.basename(title)}")
    fig.tight_layout()
    if save_as is not None:
        plt.savefig(save_as)
    plt.close()
    print(f"epochs with nsat <= 4: {round(len(bad_nsat)/len(kin_df)*100,2)} %")
    print(f"epochs with PDOP >= 5: {round(len(bad_pdop)/len(kin_df)*100,2)} %")
    print(f"epochs with wrms >= 20: {round(len(bad_wrms)/len(kin_df)*100,2)} %")

#plot_kin_results(kin_df, save_as = f"{pride_dir}/kin_results.png")
#plot_kin_results_wrms(kin_df, save_as = f"{pride_dir}/kin_results.png")



# parse the res file to make the following plots
- residuals vs elevation per sat
- residuals vs time per sat
- elevation vs time

In [6]:

def read_res_data(res_path):
    with open(res_path, "r") as res_file:
        data = []
        line = res_file.readline() #first line we can throw away
        while True: 
            if line == "":
                break
            line_data = line.split()
            if line_data[0] == 'TIM':
                seconds = float(line_data[6])
                SS = int(seconds)
                f = str(seconds - SS).split('.')[-1]
                isodate = f"{line_data[1]}-{line_data[2].zfill(2)}-{line_data[3].zfill(2)}T{line_data[4].zfill(2)}:{line_data[5].zfill(2)}:{str(SS).zfill(2)}.{f}"
                timestamp = datetime.fromisoformat(isodate)
                #read data for that epoch
                line = res_file.readline()
                line_data = line.split()
                while not line.startswith('TIM'):
                    #print(line_data)
                    prn = line_data[0]
                    line_data[3] = float(line_data[3].replace('D','E'))
                    line_data[4] = float(line_data[4].replace('D','E'))
                    row = [timestamp]+line_data
                    data.append(row)
                    line = res_file.readline()
                    if line == "":
                        break
                    line_data = line.split()
            else:
                line = res_file.readline()
    res_df = pd.DataFrame(data, columns=['date','prn','phase_res','psuedorange_res','phase_weight','psuedorange_weight',
                                          'data_status_id','elevation','azimuth','obs1','obs2','obs3','obs4'])
    res_df=res_df.set_index(['date'])#,'prn'])
    return res_df

def get_satellites_from_res(res_path):
    satellites = []
    with open(res_path, "r") as res_file:
        for i, line in enumerate(res_file):
            if "SATELLITE LIST" in line:
                line_data = line.split()
                satellites += line_data[:-2]
            if "END OF HEADER" in line:
                end_of_header = i
                break
    return satellites

# res_df = read_res_data(res_path)
# satellites = get_satellites_from_res(res_path)
# res_df = res_df[['prn','phase_res','elevation']].astype({'phase_res':'float','elevation':'float'})

In [7]:
def plot_res_v_elev(res_df,satellites, plot_title):
    fig, axs = plt.subplots(len(satellites),1, figsize=(8,len(satellites)*1.5), sharex=False)
    for i, prn in enumerate(satellites):
        tmp_df = res_df[res_df['prn']==prn]
        axs[i].scatter(tmp_df['elevation'],tmp_df['phase_res']*1000, s=2)
        axs[i].set_ylabel(prn)
        axs[i].set_ylim(-50,50)
    axs[i].set_xlabel('elevation')
    fig.suptitle(f"Phase Residuals mm vs elevation angle for {os.path.basename(plot_title)}")
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(f"{pride_dir}/res_v_elev.png")
    plt.close()
#plot_title = f"{site}_{year}_{dayOfYear}"
#plot_res_v_elev(res_df,satellites)

In [8]:
def plot_res_v_time(res_df,satellites, plot_title):
    fig, axs = plt.subplots(len(satellites),1, figsize=(8,len(satellites)*1.5), sharex=True)
    for i, prn in enumerate(satellites):
        tmp_df = res_df[res_df['prn']==prn]
        axs[i].scatter(tmp_df.index,tmp_df['phase_res']*1000, s=2)
        axs[i].set_ylabel(prn)
        axs[i].set_ylim(-50,50)
    axs[i].set_xlabel('elevation')
    fig.suptitle(f"Phase Residuals mm vs time for {os.path.basename(plot_title)}")
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(f"{pride_dir}/res_v_time.png")
    plt.close()
    

#plot_title = f"{site}_{year}_{dayOfYear}"
#plot_res_v_time(res_df,satellites)

In [181]:
def plot_elev_v_time(res_df,satellites,plot_title):
    fig, axs = plt.subplots(len(satellites),1, figsize=(8,len(satellites)*1.5), sharex=True)
    for i, prn in enumerate(satellites):
        tmp_df = res_df[res_df['prn']==prn]
        axs[i].scatter(tmp_df.index,tmp_df['elevation'], s=2)
        axs[i].set_ylabel(prn)
        axs[i].set_ylim(0,90)
    axs[i].set_xlabel('elevation')
    fig.suptitle(f"elevation vs time for {os.path.basename(plot_title)}")
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.savefig(f"{pride_dir}/elev_v_time.png")
    plt.close()
#plot_title = f"{site}_{year}_{dayOfYear}"
#plot_elev_v_time(res_df,satellites)


# Loop through days in survey, parse results, and make plots

In [201]:

# need to manually enter the dayOfYear range

kin_df = None
wrms_df = None
year = survey.split('_')[0]
for dayOfYear in range(121,131):
    print(f"{site} {survey} Day: {dayOfYear}")
    plot_title = f"{site}_{year}_{dayOfYear}"
    daily_pride_dir = f"{pride_dir}/{year}/{dayOfYear}"
    try:
        #make the daily kin_results plots, build a full dataframe of kin results for whole survey
        log_path = os.path.join(daily_pride_dir, f"log_{year}{dayOfYear}_{site}")
        kin_path = os.path.join(daily_pride_dir, f"kin_{year}{dayOfYear}_{site}")
        res_path = os.path.join(daily_pride_dir, f"res_{year}{dayOfYear}_{site}")
        get_epochs_from_log(log_path)
        if kin_df is None:
            kin_df = read_kin_data(kin_path)
            wrms_df = get_wrms_from_res(res_path)
            kin_df['wrms']=wrms_df['wrms']
            plot_kin_results_wrms(kin_df, title=plot_title, save_as=f"{daily_pride_dir}/kin_results.png")
        else:
            kin_df2 = read_kin_data(kin_path)
            wrms_df2 = get_wrms_from_res(res_path)
            kin_df2['wrms']=wrms_df2['wrms']
            plot_kin_results_wrms(kin_df2, title=plot_title, save_as=f"{daily_pride_dir}/kin_results.png")
            kin_df = pd.concat([kin_df, kin_df2])
        
    except Exception as e:
        print(e)
    try:
        #make extra daily plots
        res_path = os.path.join(daily_pride_dir, f"res_{year}{dayOfYear}_{site}")
        res_df = read_res_data(res_path)
        satellites = get_satellites_from_res(res_path)
        res_df = res_df[['prn','phase_res','elevation']].astype({'phase_res':'float','elevation':'float'})
        plot_res_v_elev(res_df,satellites,plot_title)
        plot_res_v_time(res_df,satellites,plot_title)
        plot_elev_v_time(res_df,satellites,plot_title)
    except Exception as e:
        print(e)

#make the full survey kin_results plot
plot_title = f"{site}_{survey}"
plot_kin_results_wrms(kin_df, title=plot_title, save_as=f"{pride_dir}/{year}/{survey}_kin_results.png")


NCC1 2022_A_1065 Day: 121
NCC1 2022_A_1065 Day: 122
[Errno 2] No such file or directory: '/Users/gottlieb/working/GIT/seafloor_geodesy_notebooks/notebooks/data/Pride/2022/122/res_2022122_NCC1'
NCC1 2022_A_1065 Day: 123
NCC1 2022_A_1065 Day: 124
NCC1 2022_A_1065 Day: 125
NCC1 2022_A_1065 Day: 126
NCC1 2022_A_1065 Day: 127
NCC1 2022_A_1065 Day: 128
NCC1 2022_A_1065 Day: 129
[Errno 2] No such file or directory: '/Users/gottlieb/working/GIT/seafloor_geodesy_notebooks/notebooks/data/Pride/2022/129/res_2022129_NCC1'
NCC1 2022_A_1065 Day: 130
